# MPIプログラム 課題


## 課題について

課題１、２、３がある。それぞれのGoogleFormに指定されいてる形で、colabのリンクを含めて5月30日までに提出せよ。CoursePowerに提出場所（GoogleForm）が設定されている。

### GoogleColab について2
pythonのコードの実行がブラウザ上でできるサービス。実行する際に計算リソースに割り付けられて実行する
- コード部分 左側の括弧にマウスポインタを持って行くと実行できる
- テキスト部分 マークダウン形式で文章がかける

ファイルからドライブにコピーを保存を選択し、その後は自分のドライブにコピーされるのでそれを実行していく

### C言語の実行について
- ！をつければコマンドを実行でき、C言語のコンパイルもできる
- コード部分に %%file ファイル名 と記述することで、ファイルの中身を記述することができる
- ! gcc test.c
  - test.cをコンパイルし、a.outの実行ファイルを作成する
- ! ./a.out
  - a.outを実行


## 課題１

temp1.cをコンパイルし、10プロセスで実行せよ。実行結果を値の意味も含めて説明せよ。

Colab関連補足
- ! mpicc file.c
  - mpiのプログラムをコンパイルし、実行ファイルを生成する
- ! mpiexec --allow-run-as-root --oversubscribe -np 3 ./a.out
  - これは3プロセスでmpiプログラム ./a.out を実行する


In [1]:
%%file temp1.c
#include <stdio.h>
#include <mpi.h>
int main(){
  int i, my_rank, num_procs;
  MPI_Init(NULL, NULL);
  MPI_Comm_rank(MPI_COMM_WORLD, &my_rank);
  MPI_Comm_size(MPI_COMM_WORLD, &num_procs);

  printf("numprocs:%3d my_rank:%3d \n",num_procs, my_rank);

  MPI_Finalize();
  return 0;
}

Writing temp1.c


In [9]:
! mpicc temp1.c
! mpiexec --allow-run-as-root --oversubscribe -np 10 ./a.out

numprocs: 10 my_rank:  1 
numprocs: 10 my_rank:  3 
numprocs: 10 my_rank:  4 
numprocs: 10 my_rank:  8 
numprocs: 10 my_rank:  2 
numprocs: 10 my_rank:  9 
numprocs: 10 my_rank:  0 
numprocs: 10 my_rank:  6 
numprocs: 10 my_rank:  5 
numprocs: 10 my_rank:  7 


## 課題２

1プロセスの時のinner_product.c の実行時間を詳細に分けて計測し、プログラムとしては何倍まで高速化できそうか見積もってみよ。そのため、自分で時間計測関数を挿入せよ。

scatterを読んでいるため、プロセス数がベクトルサイズの約数とならなければならないことに注意せよ。

###実行時間計測場所
メイン関数内のtimer1からtimer4まで計測する。下に書かれているように、時間計測では、MPI_Barrier関数とMPI_Wtime関数を使う。

###プログラム内容と実行時間
- あるプロセスでベクトルを乱数で初期化 （逐次処理）
  - timer1-timer2
- それを全プロセスに分散（逐次処理）
  - timer2-timer3
- 並列に内積計算（並列処理）
  - timer3-timer4





In [3]:
%%file inner_product_mpi.c
#include<stdio.h>
#include<stdlib.h>
#include<mpi.h>
void init_data(double *X, int size)
{
  int i;
  srand(10);
  for(i=0;i<size;i++) X[i]= (double)rand();
}

double inner_product2(double *data1,double  *data2, int n)
{
   double val, nrm;
   int j;
   val = 0.0;
   for(j=0;j<n;j++) val += data1[j]*data2[j];
   MPI_Allreduce(&val, &nrm, 1, MPI_DOUBLE, MPI_SUM, MPI_COMM_WORLD);
   return(nrm);
}

int main()
{
  double *X,*pX,nrm;
  int n,SIZE=10000000, my_rank, numproc;;
  MPI_Init(NULL, NULL);
  MPI_Comm_rank(MPI_COMM_WORLD, &my_rank);
  MPI_Comm_size(MPI_COMM_WORLD, &numproc);

  //---timer 1

  if(my_rank==0){
    X=(double *)malloc(sizeof(double)*SIZE);
    init_data(X, SIZE);
  }

  //---timer 2

  n=SIZE/numproc;// 割り切れる必要あり
  pX = (double*)malloc(sizeof(double) * n);
  MPI_Scatter(X, n, MPI_DOUBLE, pX, n, MPI_DOUBLE, 0, MPI_COMM_WORLD);

  //---timer 3


  nrm=inner_product2(pX, pX, n);
  if(my_rank==0) printf("length^2 is %f\n", nrm);

  //---timer 4

  MPI_Finalize();
}

Writing inner_product_mpi.c


In [4]:
! mpicc inner_product_mpi.c
! time mpiexec --allow-run-as-root --oversubscribe -np 1 a.out


length^2 is 15373394046108394074406912.000000

real	0m1.626s
user	0m0.520s
sys	0m0.194s


## 課題３

下のコードは、ランク番号の隣接するプロセス間で、整数1個を交換するプログラムである。隣接するプロセスとは、ここでは連番のプロセス（ランク番号がmy_rank-1,my_rank+1の２プロセス）を意味する。

if文の条件式を適切に設定すると、どんなプロセス数で実行しても動作するようになる。
...内を適切に修正せよ。また、２プロセス間で通信する際に時間計測を行い、プロセス間で整数1個交換するのに、どの程度の時間がかかっているか見積もってみよ。

In [5]:
%%file test.c
#include<stdio.h>
#include<stdlib.h>
#include<mpi.h>

int main()
{
  int my_rank, numproc;
  int val, rval1=-1, rval2=-1;
  MPI_Init(NULL, NULL);
  MPI_Comm_rank(MPI_COMM_WORLD, &my_rank);
  MPI_Comm_size(MPI_COMM_WORLD, &numproc);

  MPI_Status s;
  val = my_rank;

  // my_rank <--> my_rank-1 data交換
  if(...){
    MPI_Send(&val, 1,MPI_INT,my_rank-1, 0,MPI_COMM_WORLD);
    MPI_Recv(&rval1,1,MPI_INT,my_rank-1,0,MPI_COMM_WORLD,&s);
  }

  // my_rank <--> my_rank+1 data交換 >
  if(...){
    MPI_Send(&val, 1,MPI_INT,my_rank+1, 0,MPI_COMM_WORLD);
    MPI_Recv(&rval2,1,MPI_INT,my_rank+1,0,MPI_COMM_WORLD,&s);
  }

  printf("%d my_rank:%d %d\n", rval1, val, rval2);
  MPI_Finalize();
}

Writing test.c


In [6]:
! mpicc test.c
! mpiexec --allow-run-as-root --oversubscribe -np 10 a.out

test.c: In function ‘main’:
test.c:17:6: error: expected expression before ‘...’ token
   17 |   if(...){
      |      ^~~
test.c:23:6: error: expected expression before ‘...’ token
   23 |   if(...){
      |      ^~~
length^2 is 15373394046106489256411136.000000


# 時間計測方法

特定の部分の時間を計測する方法について説明する
- MPI_Wtime（）という関数がある。
- マルチプロセスで時間計測をするときには、MPI_Barrier関数を呼んで全員が揃ってからタイマーをスタートしもう一度Barrier関数を呼んでストップする。

In [10]:
%%file time_measure_mpi.c
#include<stdio.h>
#include<mpi.h>

int main()
{
  double s,e;
  int my_rank, numproc, i,j=0;
  MPI_Init(NULL, NULL);
  MPI_Comm_rank(MPI_COMM_WORLD, &my_rank);
  MPI_Comm_size(MPI_COMM_WORLD, &numproc);

  //timer 1 ----
  MPI_Barrier(MPI_COMM_WORLD);
  s=MPI_Wtime();

  for(i=0;i<10000000;i++) j++;

  //timer 2 ----
  MPI_Barrier(MPI_COMM_WORLD);
  e=MPI_Wtime();

  //timer 1,2の間の時間の表示
  if(my_rank==0) printf("%lf second \n", e-s);

  MPI_Finalize();
  return 0;
}

Overwriting time_measure_mpi.c


In [8]:
! mpicc time_measure_mpi.c
! mpirun --allow-run-as-root --oversubscribe -np 20 ./a.out

0.854425 second 
